In [ ]:
import time
from time import strftime, localtime, strptime
import OpenOPC
from psycopg2 import connect


remotehost="172.16.255.2"

try:
        
        conexion = connect(database="capem",host="localhost",user="postgres",password="root",port="5432")
        cursor = conexion.cursor()

        opcserver="ABB.AC800MC_OpcDaServer.3"
        mainTagname="Applications.CAPEM"
        tagnames="LT035_mmH2O","FT003_In","FT019_In","FT023_In",\
        "PT030_In","BI005a_Out","BI005b_Out","BI005c_Out","DPCV027_Out",\
        "DPT018_In","DPT022_In","DPT026_In","DPT027_In","FV003_Out",\
        "PT015_In","PT021_In","TI024_In","TT004_In","TT005_In","TT016_In",\
        "TT024A_In","TT024B_In","TT061_In","TT062_In","TT063_In","ZI033_In",\
        "DTIC024_Out","FT003_In_m3h","DPT012_In","FT019_In_m3h","PI001_In",\
        "DPT110_corr","DPT111_corr","VA003_In_reg","VA004_In_reg","VG002_In_reg",\
        "SC_POS_reg","SC_TESTIGO_reg","VT107_In","JT108_In","TT105_In","PT106_In",\
        "PT103_In","FT104_In","TBCe_In","FV067_Out","TT101_In",\
        "FT023_In_m3h","APsubida_reg","APbajada_reg","PTsubida_reg","PTbajada_reg"
##        #tagnames="rds.register_1","rds.register_2","rds.register_3","rds.register_4","rds.register_5"
        listTags=[mainTagname+"."+x+".Value" for x in tagnames]
        flag=1
except:
        print "El sistema no puede conectarse a la base de datos. Chequee que este funcionando"
        flag=0
sampletime = 1

while flag>0: # infinite loop kill with Ctrl-C
        j=0
        try:
                opc=OpenOPC.client()
                opc.connect(opcserver,remotehost)
                v= opc.read(listTags)

                i=0
                while i<len(tagnames):
                        
                        (name,opcvalue,opcstat,opctime) = v[i]
                        vname=name.split(".")
                        name=vname[len(vname)-2]
                        tempTime=strptime(opctime,"%m/%d/%y %H:%M:%S")
                        atime = strftime("%Y/%m/%d %H:%M:%S", localtime())  #.asctime() # current computer time as float
                        opctime=strftime("%Y/%m/%d %H:%M:%S", tempTime)
                        sqlstring = "INSERT INTO opcdata (tag, val, stat, datetime, atime) \
                        VALUES('%s', '%f', '%s', '%s', '%s')"\
                        %(name,opcvalue,opcstat,opctime,atime)
                
                        cursor.execute(sqlstring)
                        cursor.execute("commit")

                        print name + " - " + str(opcvalue) + " - " + opcstat + " - " + opctime + " - " + atime # print newest value in database

                        i=i+1

		print ""

        # end of while loop
        except:
                print "Error al conectarse con el OPC Server. Chequee que el OPC Server este funcionando"
                atime = strftime("%Y/%m/%d %H:%M:%S", localtime()) # current computer time as float
                opctime=atime
                opcstat="ERROR"
                name="/error opc./"
                sqlstring = "INSERT INTO opcdata (tag, val, stat, datetime, atime) \
                VALUES('%s', '%f', '%s', '%s', '%s')"\
                %(name,0.0,opcstat,opctime,atime)
                cursor.execute(sqlstring)
                cursor.execute("commit")

        opc.close()
        time.sleep(sampletime)
##opc.close()
cursor.close ()
conexion.close ()

